# 第6章 逻辑斯谛回归与最大熵模型
## 6.1 逻辑斯谛回归
逻辑斯谛回归模型假定随机变量X的数据分布符合逻辑斯谛回归,有:  
>逻辑斯谛回归分布函数：
$$F(x)=P(X \leq x)=\frac{1}{1+e^\left(-\frac{x-\mu}{\gamma}\right)}$$

>密度函数：
$$f(x)=F'(x)=\frac{e^\left(-\frac{x-\mu}{\gamma}\right)}{\gamma(1+e^\left(-\frac{x-\mu}{\gamma}\right))^2}$$

## 6.2 二项逻辑斯谛回归模型
<b>定义1</b>：
><b>逻辑斯谛逻辑回归模型</b>
$$P(Y=1|x) = \frac{e^\left(wx+b\right)}{1+e^\left(wx+b\right)}$$  
$$P(Y=0|x) = \frac{1}{1+e^\left(wx+b\right)}$$

其中，$Y \in \{0,1\}$为二分类的类别标签空间，$x$时数据的一条样本记录，通过上述的两个式子得到的结果就是在给定了$x$的情况下，分别得到$0$和$1$的概率值，所以，有心的读者可以发现，这个思维和朴素贝叶斯的思路很像，都是通过给定样本$x$得到不同类别标签的概率值，并选择类别概率值最大的类别作为预测类别。那~，我们为了得到参数$w$和$b$（实际可以理解为只需要求解$w$），需要做哪些工作呢？

## 6.3 如何求解逻辑斯谛回归的参数
根据6.2中的式子可以得知
$$P(Y=1|x) = \frac{e^\left(wx+b\right)}{1+e^\left(wx+b\right)} = 1 - P(Y=0|x) = \frac{1}{1+e^\left(wx+b\right)}$$

所以，假定令
$$P(Y=1|x) = \pi(x)$$
则有
$$P(Y=0|x) = 1- \pi(x)$$

此时可以直接使用似然函数求解得到w的值，其中，作为补充，这里给出似然函数的表达，即有：
$$\prod_{i=1}^N[\pi(x_i)]^y_i [1-\pi(x_i)]^{1-y_i}$$
在给定似然函数的同时，为了求解原函数的最大值，需要令原函数的导数等于0，经典方法，对上式的连乘加对数求连加，然后对$w$求导数，令等于0，即可以得到最终的$w$的参数值，
>(1)写出对数似然函数
$$L(w) = \sum_{i=1}^N[y_i \log \pi(x_i) + (1-y_i) \log(1-\pi(x_i))]$$
$$= \sum_{i=1}^N[y_i \log \frac{\pi(x_i)}{(1-\pi(x_i))} + \log(1-\pi(x_i))]$$
$$= \sum_{i=1}^N[y_i (w x) - \log(1+e^\left(w x_i\right))]$$
(2)求解最大值
注意，这里仍然是没办法直接求导得出最后的结果的，所以这里我们使用梯度下降或者拟牛顿法来求解极值。

至此，得到逻辑斯谛回归的参数。  接下来将对以上的思路进行求解并编写代码实现：

>首先定义数据$X$和$Y$

In [5]:
import numpy as np
x = np.asarray([[1,1],[1,2],[1,2],[1,1],[1,1],
      [2,1],[2,2],[2,2],[2,3],[2,3],
      [3,3],[3,2],[3,2],[3,3],[3,3]])
y = np.asarray([0,0,1,1,0,
     0,0,1,1,1,
     1,1,1,1,0])

>所以这里根据似然函数式子$L(w) = \sum_{i=1}^N[y_i (w x) - \log(1+e^\left(w x_i\right))]$计算得到的对数似然函数  
注意，这里由于需要得到$wx$的值，所以，要求$w$的参数为两位数，需要定义为一个包含两个参数值得参数向量$w = (w_1,w_2)$

In [27]:
w = np.random.rand(1,2)
print w[0,0],w[0,1]

 0.5949799605673401 0.7710303791111778


>这里可以不可以使用求导的方法来求解参数的值呢？但是这里面临着一个问题就是，参数过多，多次求解偏导等于0实际上是不现实的，处理这种高纬度参数的问题，实际上都是凸优化的通用的通法了，所以这里使用梯度下降方法来求解参数$w$

In [28]:
iterNum = 10
for iter in range(iterNum):
    print('========================iteration'+ str(iter)+'========================')
    for i in range(len(x)):
        

========================iteration0========================
========================iteration1========================
========================iteration2========================
========================iteration3========================
========================iteration4========================
========================iteration5========================
========================iteration6========================
========================iteration7========================
========================iteration8========================
========================iteration9========================
